<a href="https://colab.research.google.com/github/katemartian/Photometry_data_processing/blob/master/Photometry_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>Photometry data processing</center></h1>

If you want to use this code, please cite our Jove paper:

__Martianova, E., Aronson, S., Proulx, C.D.__ [Multi-Fiber Photometry to Record Neural Activity in Freely Moving Animal.](https://www.jove.com/video/60278/multi-fiber-photometry-to-record-neural-activity-freely-moving). _J. Vis. Exp._ (152), e60278, doi:10.3791/60278 (2019).

# Set up workspace

Run the cell (Shift+Enter or Ctrl+Enter) to mount disk if you work in Google Colab

Download photometry_functions.py file from https://github.com/katemartian/Photometry_data_processing and run the following cell 

OR

Run the code cells at the end of the notebook

Import libraries

In [1]:
# Load necessary module
import os
import doric as dr
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import json
%matplotlib widget

# Your data

Copy our example.csv file from https://github.com/katemartian/Photometry_data_processing 

OR

Use your data

In [5]:
metadata = {}
metadata['folder'] = ['H:/Data/FB_data/Dopamine/SA95/080223/']
metadata['filename'] = ['SA95_080223_NAc_0000']
# folder = 'H:/Data/FB_data/Dopamine/SA78/051223/' # Modify it depending on where your file is located
# file_name = 'SA78_NAc_051223_0000'   # Change to your data file
# folder = 'H:/Data/FB_data/Dopamine/SA78/051523/' # Modify it depending on where your file is located
# file_name = 'SA78_DLS_051523_0000'   # Change to your data file
# folder = 'H:/Data/FB_data/Dopamine/SA78/051623/' # Modify it depending on where your file is located
# file_name = 'SA78_NAc_051623_0001'   # Change to your data file
# Folder with your files
# folder = 'H:/Data/FB_data/Dopamine/SA77/051923/' # Modify it depending on where your file is located
# file_name = 'SA77_DLS_051923_0000'
folder = metadata['folder'][0]
file_name = metadata['filename'][0]
data =[]
series = ['Series0001', 'Series0002', 'Series0003']
for ser in series:
    dio01 = dr.h5read(folder + file_name + '.doric',['DataAcquisition','FPConsole','Signals',ser,'DigitalIO','DIO01'])[0]
    dio02 = dr.h5read(folder + file_name + '.doric',['DataAcquisition','FPConsole','Signals',ser,'DigitalIO','DIO02'])[0]
    dio03 = dr.h5read(folder + file_name + '.doric',['DataAcquisition','FPConsole','Signals',ser,'DigitalIO','DIO03'])[0]
    dio04 = dr.h5read(folder + file_name + '.doric',['DataAcquisition','FPConsole','Signals',ser,'DigitalIO','DIO04'])[0]
    dio_time = dr.h5read(folder + file_name + '.doric',['DataAcquisition','FPConsole','Signals',ser,'DigitalIO','Time'])[0]
    infusion = dr.find_event(dio02, dio_time)
    front    = dr.find_event(dio03, dio_time)
    back     = dr.find_event(dio04, dio_time)
    leverRetraction, leverInsertion = dr.detect_edges(dio01, dio_time)
    # Load from Channel 2: AOUT01 is reference, AOUT02 is signal
    raw_reference = dr.h5read(folder + file_name + '.doric',['DataAcquisition','FPConsole','Signals',ser,'AIN02xAOUT01-LockIn','Values'])[0]
    raw_signal    = dr.h5read(folder + file_name + '.doric',['DataAcquisition','FPConsole','Signals',ser,'AIN02xAOUT02-LockIn','Values'])[0]
    raw_time      = dr.h5read(folder + file_name + '.doric',['DataAcquisition','FPConsole','Signals',ser,'AIN02xAOUT02-LockIn','Time'])[0]
    # Load processed data
    signal        = dr.h5read(folder + file_name + '_DFF.doric',['DataProcessed','FPConsole','DFFSignals',ser,'AIN02xAOUT02-LockIn','Values'])[0]
    time          = dr.h5read(folder + file_name + '_DFF.doric',['DataProcessed','FPConsole','DFFSignals',ser,'AIN02xAOUT02-LockIn','Time'])[0]

    data_series = {'infusion': infusion, 'front': front, 'back': back, 'leverInsertion': leverInsertion, 'leverRetraction': leverRetraction, 'raw_reference': raw_reference,
                'raw_signal': raw_signal, 'raw_time': raw_time, 'signal': signal, 'time': time}
    data.append(data_series)

# all_data = []
# for i in range(len(data)):
# 	if i == 0:
# 		all_data = data[i].copy()
# 	else:
# 		for key in data[i]:
# 			all_data[key] = np.concatenate((all_data[key], data[i][key]))

# data[2]

SyntaxError: unterminated string literal (detected at line 36) (2643324099.py, line 36)

In [ ]:
data[2]

Choose from the dataframe calcium dependent and independent signals

In [4]:
# Adjust these lines depending on your dataframe
raw_reference = data[1]['raw_reference']
raw_signal = data[1]['raw_signal']
raw_time   = data[1]['raw_Time']

KeyError: 'raw_Time'

Plot the raw data

In [ ]:
fig = plt.figure(figsize=(8, 10))
ax1 = fig.add_subplot(211)
ax1.plot(raw_signal,'blue',linewidth=1.5)
ax2 = fig.add_subplot(212)
ax2.plot(raw_reference,'purple',linewidth=1.5)

# Use function get_zdFF to calculate z-dF/F

Call the function

# Analysis step by step

### Smooth

In [ ]:
# remove artifacts
idx = np.where(raw_reference > 0.1)
print(idx)
start = idx[0].min()-75
stop= idx[0].max()+25

fig = plt.figure(figsize=(8, 6))
plt.plot(raw_signal[start:stop],'blue',linewidth=1.5)
plt.plot(raw_reference[start:stop],'purple',linewidth=1.5)
plt.show()



In [ ]:
smooth_win = 20
smooth_reference = smooth_data(raw_reference, smooth_win)
smooth_signal = smooth_data(raw_signal, smooth_win)

In [ ]:
fig = plt.figure(figsize=(16, 10))
ax1 = fig.add_subplot(211)
ax1.plot(smooth_signal,'blue',linewidth=1.5)
ax2 = fig.add_subplot(212)
ax2.plot(smooth_reference,'purple',linewidth=1.5)

### Find the baseline

In [ ]:
lambd = 5e4 # Adjust lambda to get the best fit
porder = 1
itermax = 50
r_base=airPLS(smooth_reference.T,lambda_=lambd,porder=porder,itermax=itermax)
s_base=airPLS(smooth_signal,lambda_=lambd,porder=porder,itermax=itermax)

In [ ]:
fig = plt.figure(figsize=(16, 10))
ax1 = fig.add_subplot(211)
ax1.plot(smooth_signal,'blue',linewidth=1.5)
ax1.plot(s_base,'black',linewidth=1.5)
ax2 = fig.add_subplot(212)
ax2.plot(smooth_reference,'purple',linewidth=1.5)
ax2.plot(r_base,'black',linewidth=1.5)

### Remove the baseline and the beginning of the recordings

In [ ]:
remove=200
reference = (smooth_reference[remove:] - r_base[remove:])
signal = (smooth_signal[remove:] - s_base[remove:])  

In [ ]:
fig = plt.figure(figsize=(16, 10))
ax1 = fig.add_subplot(211)
ax1.plot(signal,'blue',linewidth=1.5)
ax2 = fig.add_subplot(212)
ax2.plot(reference,'purple',linewidth=1.5)

### Standardize signals

In [ ]:
z_reference = (reference - np.median(reference)) / np.std(reference)
z_signal = (signal - np.median(signal)) / np.std(signal)

In [ ]:
fig = plt.figure(figsize=(16, 10))
ax1 = fig.add_subplot(211)
ax1.plot(z_signal,'blue',linewidth=1.5)
ax2 = fig.add_subplot(212)
ax2.plot(z_reference,'purple',linewidth=1.5)

### Fit reference signal to calcium signal using linear regression

In [ ]:
from sklearn.linear_model import Lasso
lin = Lasso(alpha=0.0001,precompute=True,max_iter=1000,
            positive=True, random_state=9999, selection='random')
n = len(z_reference)
lin.fit(z_reference.reshape(n,1), z_signal.reshape(n,1))

In [ ]:
print(lin.coef_)
print(lin.intercept_)

In [ ]:
fig = plt.figure(figsize=(16, 8))
ax1 = fig.add_subplot(111)
ax1.plot(z_reference,z_signal,'b.')
ax1.plot(z_reference,z_reference_fitted, 'r--',linewidth=1.5)

### Align reference to signal

In [ ]:
z_reference_fitted = lin.predict(z_reference.reshape(n,1)).reshape(n,)

In [ ]:
fig = plt.figure(figsize=(16, 8))
ax1 = fig.add_subplot(111)
ax1.plot(z_signal,'blue')
ax1.plot(z_reference_fitted,'purple')

### Calculate z-score dF/F

In [ ]:
zdFF = (z_signal - z_reference_fitted)

In [ ]:
fig = plt.figure(figsize=(16, 8))
ax1 = fig.add_subplot(111)
ax1.plot(zdFF,'black')

# Contact us

If you have any questions please contact us: ekaterina.martianova.1@ulaval.ca

# Functions

Run (Shift+Enter or Ctrl+Enter) the following 3 code cells in order to use functions.


## z-score dF/F

In [ ]:
'''
get_zdFF.py calculates standardized dF/F signal based on calcium-idependent 
and calcium-dependent signals commonly recorded using fiber photometry calcium imaging

Ocober 2019 Ekaterina Martianova ekaterina.martianova.1@ulaval.ca 

Reference:
  (1) Martianova, E., Aronson, S., Proulx, C.D. Multi-Fiber Photometry 
      to Record Neural Activity in Freely Moving Animal. J. Vis. Exp. 
      (152), e60278, doi:10.3791/60278 (2019)
      https://www.jove.com/video/60278/multi-fiber-photometry-to-record-neural-activity-freely-moving

'''

def get_zdFF(reference,signal,smooth_win=10,remove=200,lambd=5e4,porder=1,itermax=50): 
  '''
  Calculates z-score dF/F signal based on fiber photometry calcium-idependent 
  and calcium-dependent signals
  
  Input
      reference: calcium-independent signal (usually 405-420 nm excitation), 1D array
      signal: calcium-dependent signal (usually 465-490 nm excitation for 
                   green fluorescent proteins, or ~560 nm for red), 1D array
      smooth_win: window for moving average smooth, integer
      remove: the beginning of the traces with a big slope one would like to remove, integer
      Inputs for airPLS:
      lambd: parameter that can be adjusted by user. The larger lambda is,  
              the smoother the resulting background, z
      porder: adaptive iteratively reweighted penalized least squares for baseline fitting
      itermax: maximum iteration times
  Output
      zdFF - z-score dF/F, 1D numpy array
  '''
  
  import numpy as np
  from sklearn.linear_model import Lasso

 # Smooth signal
  smooth_reference = smooth_signal(reference, smooth_win)
  smooth_signal = smooth_signal(signal, smooth_win)
  
 # Remove slope using airPLS algorithm
  r_base=airPLS(smooth_reference,lambda_=lambd,porder=porder,itermax=itermax)
  s_base=airPLS(smooth_signal,lambda_=lambd,porder=porder,itermax=itermax) 

 # Remove baseline and the begining of recording
  smooth_reference = (smooth_reference[remove:] - r_base[remove:])
  smooth_signal = (smooth_signal[remove:] - s_base[remove:])   

 # Standardize signals    
  smooth_reference = (smooth_reference - np.median(smooth_reference)) / np.std(smooth_reference)
  smooth_signal = (smooth_signal - np.median(smooth_signal)) / np.std(smooth_signal)
  
 # Align reference signal to calcium signal using non-negative robust linear regression
  lin = Lasso(alpha=0.0001,precompute=True,max_iter=1000,
              positive=True, random_state=9999, selection='random')
  n = len(smooth_reference)
  lin.fit(smooth_reference.reshape(n,1), smooth_signal.reshape(n,1))
  smooth_reference = lin.predict(smooth_reference.reshape(n,1)).reshape(n,)

 # z dFF    
  zdFF = (smooth_signal - smooth_reference)
 
  return zdFF

## Smooth signal

In [ ]:
def smooth_data(x,window_len=10,window='flat'):

    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    The code taken from: https://scipy-cookbook.readthedocs.io/items/SignalSmooth.html
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
                'flat' window will produce a moving average smoothing.

    output:
        the smoothed signal        
    """

    import numpy as np

    if x.ndim != 1:
        raise(ValueError, "smooth only accepts 1 dimension arrays.")

    if x.size < window_len:
        raise(ValueError, "Input vector needs to be bigger than window size.")

    if window_len<3:
        return x

    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise(ValueError, "Window is one of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")

    s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]

    if window == 'flat': # Moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')

    y=np.convolve(w/w.sum(),s,mode='valid')

    return y[(int(window_len/2)-1):-int(window_len/2)]

## airPLS algorithm

Original code of airPLS

In [ ]:
'''
airPLS.py Copyright 2014 Renato Lombardo - renato.lombardo@unipa.it
Baseline correction using adaptive iteratively reweighted penalized least squares

This program is a translation in python of the R source code of airPLS version 2.0
by Yizeng Liang and Zhang Zhimin - https://code.google.com/p/airpls

Reference:
Z.-M. Zhang, S. Chen, and Y.-Z. Liang, Baseline correction using adaptive iteratively 
reweighted penalized least squares. Analyst 135 (5), 1138-1146 (2010).

Description from the original documentation:
Baseline drift always blurs or even swamps signals and deteriorates analytical 
results, particularly in multivariate analysis.  It is necessary to correct baseline 
drift to perform further data analysis. Simple or modified polynomial fitting has 
been found to be effective in some extent. However, this method requires user 
intervention and prone to variability especially in low signal-to-noise ratio 
environments. The proposed adaptive iteratively reweighted Penalized Least Squares
(airPLS) algorithm doesn't require any user intervention and prior information, 
such as detected peaks. It iteratively changes weights of sum squares errors (SSE) 
between the fitted baseline and original signals, and the weights of SSE are obtained 
adaptively using between previously fitted baseline and original signals. This 
baseline estimator is general, fast and flexible in fitting baseline.


LICENCE
This program is free software: you can redistribute it and/or modify
it under the terms of the GNU Lesser General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU Lesser General Public License for more details.

You should have received a copy of the GNU Lesser General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>
'''

import numpy as np
from scipy.sparse import csc_matrix, eye, diags
from scipy.sparse.linalg import spsolve

def WhittakerSmooth(x,w,lambda_,differences=1):
    '''
    Penalized least squares algorithm for background fitting
    
    input
        x: input data (i.e. chromatogram of spectrum)
        w: binary masks (value of the mask is zero if a point belongs to peaks and one otherwise)
        lambda_: parameter that can be adjusted by user. The larger lambda is,  the smoother the resulting background
        differences: integer indicating the order of the difference of penalties
    
    output
        the fitted background vector
    '''
    X=np.matrix(x)
    m=X.size
    i=np.arange(0,m)
    E=eye(m,format='csc')
    D=E[1:]-E[:-1] # numpy.diff() does not work with sparse matrix. This is a workaround.
    W=diags(w,0,shape=(m,m))
    A=csc_matrix(W+(lambda_*D.T*D))
    B=csc_matrix(W*X.T)
    background=spsolve(A,B)
    return np.array(background)

def airPLS(x, lambda_=100, porder=1, itermax=15):
    '''
    Adaptive iteratively reweighted penalized least squares for baseline fitting
    
    input
        x: input data (i.e. chromatogram of spectrum)
        lambda_: parameter that can be adjusted by user. The larger lambda is,
                 the smoother the resulting background, z
        porder: adaptive iteratively reweighted penalized least squares for baseline fitting
    
    output
        the fitted background vector
    '''
    m=x.shape[0]
    w=np.ones(m)
    for i in range(1,itermax+1):
        z=WhittakerSmooth(x,w,lambda_, porder)
        d=x-z
        dssn=np.abs(d[d<0].sum())
        if(dssn<0.001*(abs(x)).sum() or i==itermax):
            if(i==itermax): print('WARING max iteration reached!')
            break
        w[d>=0]=0 # d>0 means that this point is part of a peak, so its weight is set to 0 in order to ignore it
        w[d<0]=np.exp(i*np.abs(d[d<0])/dssn)
        w[0]=np.exp(i*(d[d<0]).max()/dssn) 
        w[-1]=w[0]
    return z
